In [2]:
#Carregar Bibliotecas 
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from dataclasses import make_dataclass


In [3]:
#Ler o ficheiro

df_sz = 11800

df=pd.read_csv('final.csv',header=None, parse_dates=[2], engine='python').head(df_sz)
df.columns =['Serial','Sales','Date']



print(df.info())
print(df.describe())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11800 entries, 0 to 11799
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Serial  11800 non-null  object        
 1   Sales   11800 non-null  int64         
 2   Date    11800 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 276.7+ KB
None
              Sales
count  11800.000000
mean       4.024831
std        4.876157
min        1.000000
25%        1.000000
50%        3.000000
75%        5.000000
max      120.000000


In [4]:
#Retira os elementos únicos de uma lista 

def unique(trends):
 
    output = []
    for x in trends:
        if x not in output:
            output.append(x)
    return output





In [5]:
#Criar vetor p

serial=list(df['Serial'])
serial=unique(serial)

serial.sort()


#Número de produtos
#print(serial)



interval=pd.date_range(start="2018-01-01",end="2020-12-31")
days=len(interval)
zeros=np.zeros(days)

In [6]:
#Vendas totais de um artigo


X = df.groupby(['Serial','Date'])['Sales'].sum()

X.columns =['Serial','Sales','Date']



Y=X.loc['8-716-011-453-AAP']
Y.index

#plt.plot(Y)

total_data = pd.DataFrame(zip(interval,zeros),columns=['Date','Sales'])


#Meter os zeros lá dentro
for t in Y.index:
    total_data.loc[total_data["Date"] == t, "Sales"] = Y[t]

#print(Y)
#total_data[total_data["Sales"] != 0]




In [7]:
total_data

,Date,Sales
0,2018-01-01,0.0
1,2018-01-02,0.0
2,2018-01-03,0.0
3,2018-01-04,0.0
4,2018-01-05,0.0
...,...,...
1091,2020-12-27,0.0
1092,2020-12-28,0.0
1093,2020-12-29,0.0
1094,2020-12-30,0.0


In [8]:
#Treinar

df=total_data
index_train=int(np.ceil(len(total_data)*0.75))
print(index_train)


#Split the dataset
# 75% and 25%

train_df=total_data.iloc[0:index_train, 1]

test_df=total_data.iloc[index_train:,1]  


print(train_df.shape)
print(test_df.shape)


822
(822,)
(274,)


In [9]:
scaler = MinMaxScaler(feature_range=(-1, 1))
dataset_train = scaler.fit_transform(train_df.values.reshape(-1,1))
dataset_test = scaler.transform(test_df.values.reshape(-1,1))
dataset_train=np.reshape(dataset_train,len(dataset_train))
dataset_test=np.reshape(dataset_test,len(dataset_test))

In [10]:
def split_series(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [11]:
n_past=3
n_features=1

In [12]:
X_train, y_train = split_series(dataset_train,n_past)


In [13]:
X_train.shape

(819, 3)

In [14]:
X_train, y_train = split_series(dataset_train,n_past)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))


X_test, y_test = split_series(dataset_test,n_past)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))


In [15]:
X_test.shape

(271, 3, 1)

In [20]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, n_past)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)
model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported